## MLflow's Model Registry

In [6]:
from mlflow.tracking import MlflowClient
from pprint import pprint

# The project we already created URL
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [7]:
# Create a client
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# client.list_experiments()
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1685474815976,
    'description': None,
    'last_updated_timestamp': 1685474827283,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1685474827283, current_stage='None', description=None, last_updated_timestamp=1685474827283, name='nyc-taxi-regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>],
    'name': 'nyc-taxi-regressor',
    'tags': {}}
{   'aliases': {},
    'creation_timestamp': 1685471453919,
    'description': 'The NYC taxi data',
    'last_updated_timestamp': 1685544614239,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1685471453977, current_stage='Production', description='', last_updated_timestamp=1685543202381, name='nyc_taxi_regressor', run_id='ae4

In [8]:
client.create_experiment(name="my-experiment")
# The 4th one in mlflow

'4'

In [18]:
# Check the latest versions for the experiment with id `1`...

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [19]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 27fb697e569043bfad5f3cc4542f3898, rmse: 6.3024


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [17]:
# The best rmse
run_id = "af17d01599484b8f81b6b98f7132c76c"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")

Registered model 'nyc_taxi_regressor' already exists. Creating a new version of this model...
2023/05/31 10:50:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc_taxi_regressor, version 5
Created version '5' of model 'nyc_taxi_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685544614239, current_stage='None', description=None, last_updated_timestamp=1685544614239, name='nyc_taxi_regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [10]:
model_name = "nyc_taxi_regressor"
# change from list_registered_model() to get_registered_model()
client.get_registered_model(name="nyc_taxi_regressor")

<RegisteredModel: aliases={}, creation_timestamp=1685471453919, description='The NYC taxi data', last_updated_timestamp=1685542888122, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685542888122, current_stage='None', description=None, last_updated_timestamp=1685542888122, name='nyc_taxi_regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>,
 <ModelVersion: aliases=[], creation_timestamp=1685474934792, current_stage='Staging', description=None, last_updated_timestamp=1685474995095, name='nyc_taxi_regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, t

In [15]:
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 3, stage: Staging
version: 4, stage: None


In [16]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685474934792, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-05-31', last_updated_timestamp=1685543219623, name='nyc_taxi_regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685474934792, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-05-31', last_updated_timestamp=1685543117980, name='nyc_taxi_regressor', run_id='af17d01599484b8f81b6b98f7132c76c', run_link=None, source='/Users/amberm/PycharmProjects/MLcamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/af17d01599484b8f81b6b98f7132c76c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [22]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [23]:
df = read_dataframe('./data/green_tripdata_2021-03.parquet')

In [1]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

NameError: name 'client' is not defined

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

In [41]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 139 ms, sys: 44.6 ms, total: 183 ms
Wall time: 447 ms


{'rmse': 6.659623830022514}

In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.94 s, sys: 216 ms, total: 7.16 s
Wall time: 7.28 s


{'rmse': 6.881555517147188}

In [43]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1652971637398, current_stage='Production', description='The model version 4 was transitioned to Staging on 2022-05-19', last_updated_timestamp=1652972763255, name='nyc-taxi-regressor', run_id='b8904012c84343b5bf8ee72aa8f0f402', run_link=None, source='./mlruns/1/b8904012c84343b5bf8ee72aa8f0f402/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>